# This project contains sensitive information. To protect the client's identity, I will not be reading the csv file that will allow the cells below to run. For more information, please refer to the README.md

I wanted to have more elegant code, but I figured brute force would be easier to understand. With brute force, you'll also be able to understand what is happening step by step, as well as identify any errors. 

Every sheet in the final excel file will have a different set of information. The requested format was to have multiple "big picture" sheets, and then focus in on some of the desired issues. This is acquired by assigning conditions to variables, and then appending those variables (dataframes) to seperate excel sheets. 

Common errors to check for are as follows:
Make sure when calling for columns, that there are no unnecessary spaces, like 'First  Name' (which has two spaces between First and Name. 

Ensure that the dates are stored in MM/DD/YYYY format. This needs to be the format throughout the file in order to do proper analysis. 


Few tips: 
Shift + Enter runs the current cell and moves you down to the next one. 

This notebook is formatted using Markdowns (used to explain and format) and code cells (used to run code).

This notebook has everything seperated. If you'd like to just run one cell, please refer to the other notebook, where everything is in a function. If there is an error with that function, please refer to this notebook and identify where the issue lays -- if you can't figure out the error code in the other notebook. 

Ensure that your file names are formatted for windows. ('\\Users\\filelocation\\filename.csv). Ensure that the append_to_df function is also formatter for windows ('\\Users\\filelocation\\filename.xlsx).

We need to import two libraries in order to use some of the utilities that they provide. 
These utilities will prove to be useful in cleaning our data, as you will see below. 

In [9]:
import pandas as pd
import numpy as np

We are now creating a function that allows us to transfer the information that we have into excel spreadsheets. When calling the function, you must make sure that all of your arguments are accounted for in order for it to work properly.

filename - must be an actual location on the device. Since it is asking for the filename, don't forget to end the location with .xlsx to represent excel files. 

df - represents the DataFrame that you'd like to transfer to the excel sheet. We are saving all of our DataFrames into variables, so this section would require you to only input the variable name.

sheet_name - asks for the name that you'd like the sheet to have. If you have two instances of the same sheet, both records that you specify will be stored there. 
For example:
append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', INVALID_SOCIAL, sheet_name='Invalid_Social')
append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', NO_SOCIAL, sheet_name = 'Invalid_Social')

In this scenario, both variables INVALID_SOCIAL and NO_SOCIAL are being stored in the file Vendor_List_Exceptions.xlsx, and they will both be stored in the sheet name Invalid Social. 
For the sake of simplicity and so that everyone who receives this file can understand properly, each variable will be assigned to a different sheet. 

startrow and truncate_sheet are not going to be used, although it could be. These have default values (None and False respectively), meaning you don't need to input anything for them. Similarly, sheet_name has a default value as well (Sheet1), meaning if you don't input a sheet name, it'll store the information into Sheet1. 

In [10]:
def append_df_to_excel(filename, df, sheet_name = 'Sheet1', startrow = None, 
                       truncate_sheet = False, **to_excel_kwargs):
    from openpyxl import load_workbook
    
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')
        
    writer = pd.ExcelWriter(filename, engine = 'openpyxl')
    
    try:
        writer.book = load_workbook(filename)
        if startrow is None and sheet_name in writer.book.sheetnames:
            startrow = writer.book[sheet_name].max_row
        
        if truncate_sheet and sheet_name in writer.book.sheetnames:
            idx = writer.book.sheetnames.index(sheet_name)
            writer.book.remove(writer.book.worksheets[idx])
            writer.book.create_sheet(sheet_name, idx)
            
        writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
        
    except FileNotFoundError:
        pass
    
    if startrow is None:
        startrow = 0
        
    df.to_excel(writer, sheet_name, startrow = startrow, **to_excel_kwargs)
    
    writer.save()

The cell below is what we're using to read the initial excel file that you will receive from Gili's department. By storing that excel sheet into a variable, we can then begin cleaning it and doing our analysis. 

In the first cell we names the pandas library as pd. The ability to read_csv is exclusive to the pandas library, so we have to call it, and then call what we want it to do. pd.read_csv does exactly that. We're going to write the directory of where the file is located, the name of the file, and the type of file it is. read_csv can only read csv files, which is very important. Csv files are great because it removes a lot of formatting such as merged cells, and really simplifies the data. 

This specific instance is calling the NYF file. Note that you will need to change this to read the JASA and SELFHELP files as well. 

'#' is used to comment out code, meaning that code selection won't run. 

If for some reason, the regular pd.read_csv file is not working, don't be alarmed. Sometimes the encoding needs to be changed in order for it to be read. If that is the case, use the commented code instead of the initial one. 

In [11]:
#VENDOR_LIST = pd.read_csv('/Users/filelocation/filename.csv')
VENDOR_LIST = pd.read_csv('/Users/filelocation/filename.csv', encoding='latin1')

This new DataFrame MONTH_ERROR might be a bit tricky to understand. 

MONTH_ERROR is the variable that we are saving the end result of this whole thing to. 

VENDOR_LIST['More than 6 months since status date?'] will show you the values of that one column. If you just run that part, you'll get a result in a format called a Series. It is super simplified and only shows that one column. If you'd like to see that same result in the form of a dataframe, you would need to write it in the format
VENDOR_LIST[VENDOR_LIST['More than 6 months since status date?']]

.apply() is a popular function that applies what is in the parentheses to what's on the left hand side of the function. 

lambda x: x == '#VALUE!' is meant to select all records in the column that are equal to '#VALUE!'. Notice that when we are saving the information to a variable, it is only ONE = sign. If we are using a condition as a check, we are using TWO = signs. 

As I mentioned before, MONTH_ERROR will have all of the records where in the column 'More than 6 months since status date?', there is an entry that says '#VALUE!'

We don't touch MONTH_ERROR for the rest of this project, aside from when we export it later. It's purpose of finding the records that say '#VALUE!' has been fulfilled and we don't need it anymore. 

In [12]:
MONTH_ERROR = VENDOR_LIST[VENDOR_LIST['More than 6 months since status date?'].apply(lambda x: x == '#VALUE!')]

Here is where we begin manipulating the variable VENDOR_LIST, which is what has all of the information from the excel sheet that we imported. 

On the variable VENDOR_LIST, we are using the .replace() function, which allows us to change certain values into something else. In this case, we are replacing all of the different ways that people write None and empty spaces, as well as all of the records that say '#VALUE!' into something called np.nan. This is using the numpy library that we imported above. np.nan is the equivalent of a Null or NaN value. The word Null or NaN is not the same as the np.nan value, since Null or NaN can be interpreted as a word. the np.nan value officially makes it into an empty value. 

We store all of the types of writing out None in brackets [], to ensure that every one of those words are processed. 
Finally, you see that we say inplace = False. If this were True, then the change would happen immediately to the VENDOR_LIST variable. Since we are saving this new version of the DataFrame to the VENDOR_LIST_FULL variable, it isn't necessary for inplace to be True. This is somewhat safer too, in case if there's a mistake somewhere, you still have your original verson of VENDOR_LIST. If inplace were True, you would not have that original version of VENDOR_LIST, unless if you reran the cell above that was used to create VENDOR_LIST. 

In [13]:
VENDOR_LIST_FULL = VENDOR_LIST.replace(['None', 'nan', '', ' ', 'NONE',
                                        'na', 'NA', 'unk', 'unknown',
                                        'UNKNOWN', 'UNK', '#VALUE!'], np.nan, inplace = False)

DataFrame['Column'] is general format to call for a specific column as a series. In the instance below, we are getting the column 'More than 6 months since status date?' from the DataFrame VENDOR_LIST_FULL. If we were to solely run DataFrame['Column'], you'll get a Series, as I mentioned before. In this case however, we only want to change the values of that column to something else, so we don't need to format it as a DataFrame.

Notice that there is an equal sign in the middle of the code. This exists because we are changing the values of that column. We are making the new version of 'More than 6 months since status date?' equal the same values, but as a float type. Originally, they were of type object, which is okay, but we need it in a number format in order for us to do some of our future analysis. As you might have guessed, .astype(type) allows you to change the fundamental type of the data. 

If this is confusing, try to think of it like this:

x = 123

x = x.astype(float), where x starts off as the number 123, and at the end of it becomes 123.0, and we are storing 123.0 to the variable x

'#VALUE!' would normally be represented as a string, but since earlier we specified that we wanted it to be seen as a NaN value, we won't have any issues with the conversion to float.

In [14]:
VENDOR_LIST_FULL['More than 6 months since status date?'] = VENDOR_LIST_FULL['More than 6 months since status date?'
                                                                            ].astype(float)

In the cell below, we are making a new variable names VALUE_COUNT. This will not be a DataFrame. 
We are calling for the column 'Living/Domicile Statue' (becasue remember DataFrame['Column'] is used to focus on specific columns) and are using the .value_counts() function to see the breakdown for that column.

This will explain to you how many clients are listed as Deceased, Incarcerated, Nursing Home, etc. 

In [15]:
VALUE_COUNT = VENDOR_LIST['Living/Domicile Status'].value_counts()

Similar to the cell above, we are using the .value_counts() function on a column again. This time, we are checking for the various List Types (Discharge Processing / Active).

.value_counts() is also useful to check to see if there are any common typos with entries. For example, the user can input multiple records of active (lowercase a) instead of Active, and .value_counts() can help you realize how many times this happens.

In [16]:
LIST_TYPE = VENDOR_LIST_FULL['List Type'].value_counts()

Missing_Demo is going to cover a large amount of information. 

Notice that this is in DataFrame format (DataFrame[DataFrame[]]).
Here, we are using the .loc() function, to find every instance of values that .isna() (is NaN) in .any(axis = 1) (in any column. Axis = 1 represents columns, Axis = 0 represents rows) in a certain range. 

.loc[: , :'Date Since'] uses a technique names slicing to select 
[EVERY ROW (shown with the colon) , EVERY COLUMN (until -- also shown with the colon) 'Date Since']
Normally is it [row, column]. With slicing, : for row will focus on selecting every row. the comma seperates row and column. The next : will focus on selecting every column. By placing a column name at the right hand side of the colon, we are indicating that we want every column UP TO 'Date Since'. 

VENDOR_LIST_FULL[VENDOR_LIST_FULL.loc[:, :'Date Since']] can be used to solely select those values. We then use the functions .isna() and .any() to add more conditions. 

.isna() checks for all values that are empty. This however needs a bit of specification. It could check for rows or columns that are entirely empty, or rows and columns that have any number of empty values in them. 
This is why we use .any(), to emphasize that we want the row and column with any number of empty values, and we want it done column wise, not row wise (axis = 1). 

If you open the original excel sheet, you'd see that there are two sections, a blue and orange section. This one piece of code will select every entry that has an empty value in the entire blue section, indicating that the Vendors who send us these forms need to add information to these clients. 

In [17]:
Missing_Demo = VENDOR_LIST_FULL[VENDOR_LIST_FULL.loc[:, :'Date Since'].isna().any(axis = 1)] 

In the original excel sheet, Social Security Numbers follow the XXX-XX-XXXX format. This is a total of 11 characters, including the dashes. INVALID_SOCIAL checks for every entry where the length of characters is less than 11 characters. In order to do this, we must first look at the information as if it were a string, and not an object (which is the data type that it currently is. To get a better look at this, run VENDOR_LIST_FULL.info(), and you'll see what type of data each column is). 

We are converting every entry in the column ['S.S.#.'] into a string, to then check if there are less than 11 characters in the entry. Conveniently, this will also highlight entries that have 'No Social' listed, since No Social is only 9 characters. This method can not select cases that have NO SOCIAL, since we can't convert an empty value into a string. We need to do something else to check for empty socials. 

In [18]:
INVALID_SOCIAL = VENDOR_LIST_FULL[VENDOR_LIST_FULL['S.S.#.'].str.len() < 11]

Here we use .loc again to find every entry in the 'S.S.#.' column in the VENDOR_LIST_FULL DataFrame that .isnull(). By following the VENDOR_LIST_FULL[VENDOR_LIST_FULL[]] format, we are saving the new variable as a DataFrame, saving all of the other columns of that DataFrame, allowing the user to also see information, such as Name, DOB, etc. 

.isnull() and .isna() are essentially the same thing. They do the same thing, so don't worry about the difference. It is really a matter of convenience and preference. 

In [21]:
NO_SOCIAL = VENDOR_LIST_FULL.loc[VENDOR_LIST_FULL['S.S.#.'].isnull()]

LDS_DECEASED is in DataFrame format, and it checks for all of the clients who have Deceased for their Living/Domicile Status. Notice that we are using == this time around, because we are checking for a condition. This can also be done using .loc, but again, its all about preference. 

In [22]:
LDS_DECEASED = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status']=='Deceased']

There will be instances where the cells of code will build off of earlier cells. In this case, we are building off of the LDS_DECEASED DataFrame that we made, and checking for another criteria -- if the column Current Discharge Status has any records that say Submit D/C Request to HRA. Code is usually split up into section to ensure that whoever looks over it can understand what is happening.

In [23]:
LDS_DECEASED_SUB = LDS_DECEASED[LDS_DECEASED['Current Discharge Status'] == 'Submit D/C Request to HRA']

Similar to LDS_DECEASED_SUB, this next cell is building off of LDS_DECEASED. Also, similar to Missing_Demo from above, we are utilizing a mixture of .loc and .isna(). Not similar to Missing Demo, we are using .all(axis = 1) instead of .any(axis = 1). 

As a reminder, in the DataFrame LDS_DECEASED, we are locating all of the rows (indicated by the first colon). 
Then, using slicing, we are specifying that we also want every column between 'Petition Date' and 'Vendor's Previous Comments'
With the use of the .isna() function, we are looking for NaN values. 
To build on that, with the .all(axis = 1) function, we are looking for all instances that the entire record is empty, column-wise (represented by the axis = 1).

If you look on the excel sheet, it'll be the equivalent of every record of a deceased client that has columns Z to AE empty (which are where you can find all of the Discharge information. These are clients that should have some type of discharge information, given that they're deceased. If they have no information, the Vendor's need to update this ASAP.

In [24]:
LDS_NO_DISCHARGE = LDS_DECEASED[LDS_DECEASED.loc[: , 'Petition Date':"Vendor's Previous Comments"
                                                ].isna().all(axis = 1)]

In the following cell, we are making a few Lists. Lists are exclusive to Python and are excellent at holding different data types. In this case, we are using it to hold various strings. 

When the Vendors are filling out their excel files, they are supposed to follow through and select the information via drop down menus. Unfortunately, this is not always the case, and they tend to either copy and paste information, use VLOOKUP (which removes formatting) or manually input the information that is needed. As a result, there will be the occassional typo for certain fields. We will be using the Lists below to properly select the correct wording, as well as select the cases that don't have the correct wording. 

You'll see some \ in the cell below. This just signifies that the information will go onto the next line. This is used to make it look a bit neater. 

The Lists below are meant to show specific presentations. 
Current_Discharge Status shows the proper Current Discharge Statuses, following the drop down menus on the excel sheet.
Living_Domicile_Status shows the proper Living Domicile Status in accordance to the drop down menus.
L_D_S is meant to make a list of the types of statuses that represent Institutionalized Clients.
LDStatus focuses on all Living and Domicile Statuses not including Resides in the Community.
LDS_WO_DEAD focuses on all Living and Domicile Statuses not including Deceased.

In [26]:
Current_Discharge_Status = ['Clt back in the community', 'Clt scheduled to return to the community within 6 months of admission',\
                            'Clt returned back to NYC', 'Successor guardian appointed FA to be submitted', 'Submit D/C Request to HRA',\
                            'Judge denied D/C', 'Successor guardian not yet commissioned', 'Vendor not seeking discharge', 'Clt discharged',\
                            'Pending Court Approval']
    
Living_Domicile_Status = ['Deceased', 'Adult Home', 'Assisted Living', 'Supportive Housing', 'Hospital', 'Shelter',\
                          'Nursing Home-Perm', 'Nursing Home-Temp', 'Street Homeless', 'Incarcerated',\
                          'Resides in Community', 'Whereabouts unknown', 'Out of Jurisdiction', 'Substance abuse facility',\
                          'OPWDD facility', 'Congregate Care Facility']
    
L_D_S = ['Adult Home', 'Assisted Living', 'Supportive Housing', 'Hospital', 'Shelter',\
                          'Nursing Home-Perm', 'Nursing Home-Temp', 'Incarcerated','OPWDD facility', 'Congregate Care Facility']
    
LDStatus = ['Deceased', 'Adult Home', 'Assisted Living', 'Supportive Housing', 'Hospital', 'Shelter',\
                          'Nursing Home-Perm', 'Nursing Home-Temp', 'Street Homeless', 'Incarcerated',\
                          'Whereabouts unknown', 'Out of Jurisdiction', 'Substance abuse facility',\
                          'OPWDD facility', 'Congregate Care Facility']
        
LDS_WO_DEAD = ['Adult Home', 'Assisted Living', 'Supportive Housing', 'Hospital', 'Shelter',\
                          'Nursing Home-Perm', 'Nursing Home-Temp', 'Street Homeless', 'Incarcerated',\
                          'Resides in Community', 'Whereabouts unknown', 'Out of Jurisdiction', 'Substance abuse facility',\
                          'OPWDD facility', 'Congregate Care Facility']

If there are cells with multiple variables in it, chances are, the final result will build off of the prior variables in that cell. If that is the case, I will put them all together so that we can know they are connected.

Now is when we begin to focus on the lists that we have above.
LDSWODEAD is going to be in DataFrame format, and using the .isin() function, we are checking if the entries for 'Living/Domicile Status' match what is in the List. 

For example, if we have the following entries:
Adult Home
ADULT HOME
Assisted Living
Shelter

And we then run the code below, we will get the result:
Adult Home
Assisted Living
Shelter
because we don't have ADULT HOME in our list. Since it is in DataFrame format, it will carry over the other information as well, such as client name and SSN.

The next session makes the variable HRA_DECEASED_NULL, which uses information that we covered before. We are checking the column 'HRA Approved (Y/N)' for empty values using the .isna() function.

In [27]:
LDSWODEAD= VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'].isin(LDS_WO_DEAD)]
HRA_DECEASED_NULL = LDSWODEAD[LDSWODEAD['HRA Approved (Y/N)'].isna()]

Similar to the LDSWODEAD cell, we are using the Living_Domicile_Status list to filter out the VENDOR_LIST_FULL DataFrame. In the cell below, we are doing something different though. The ~ is meant to do the opposite of what is indicated. 

In the prior cell, we were checking for records that meet the criteria of the List. 
In the cell below, we are checking for records that DON'T meet that criteria, which is signified by the ~.

Using our earlier example:
Adult Home
ADULT HOME
Assisted Living
Shelter

We will get the result:
ADULT HOME

In [28]:
LDS_ERROR = VENDOR_LIST_FULL[~VENDOR_LIST_FULL['Living/Domicile Status'].isin(Living_Domicile_Status)] 

Using the same format as the cell above, CDS_ERROR is checking for cases that are not properly recorded, using the drop down selections that are specified in the Current_Discharge_Status list.

In [29]:
CDS_ERROR = VENDOR_LIST_FULL[~VENDOR_LIST_FULL['Current Discharge Status'].isin(Current_Discharge_Status)] 

Following the same format, we're checking for cases in the Living/Domicile Status column that are in the L_D_S list. Similar example to LDSWODEAD, just checking for different criteria. 

In [30]:
LDS_removed = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'].isin(L_D_S)]

The following section is building off of LDS_removed. 
LDS_removed focuses on all of the cases that are institutionalized. 
We are now checking the status date of these institutionalized cases. Since we already changed 'More than 6 months since status date?' into a float value, we can use comparisons such as greater than or less than to filter it even more. We are checking for cases that are greater than 180 days AND less than 365 days. When using the & operator, we need to repeat our area of focus, which is why we rewrite LDS_removed['More than 6 months since status date?'].

Once we have that selection, we are using .loc and slicing like in earlier cells, and are looking for records that have empty values for the discharge section, columnwise. 

If a client is institutionalized, and have been in that institution for more than 6 months, yet less than a year, they should most likely be eligible for discharge, so we'd like to check the status of that. If they are missing discharge information (Petition Date : Vendor's Previous Comments -- Alternatively, columnz Z-AE on excel), it means there is no discharge processing, and we'd like to know why. 

In [31]:
LDS180 = LDS_removed[(LDS_removed['More than 6 months since status date?'] > 180) & (LDS_removed['More than 6 months since status date?'] < 365)]
LDS180_FOCUS = LDS180[LDS180.loc[: , 'Petition Date':"Vendor's Previous Comments"].isna().all(axis = 1)]

In the cell below, we are doing the same thing we did previously, but we are checking for cases that are more than 365 days, because these are extreme cases that should be checked immediately. 

In [32]:
LDS365 = LDS_removed[LDS_removed['More than 6 months since status date?'] >= 365]
LDS365_FOCUS = LDS365[LDS365.loc[: , 'Petition Date':"Vendor's Previous Comments"].isna().all(axis = 1)]

Using the lists, we are filtering the cases that were input properly. Once we have all of those records, we are checking if any of the 'Date Since' values are empty. 

LDStatus represents all of the cases that are not Residing in the Community. These should all have a Data Since specification (for example, what was the Date Since the client was institutionalized). If they don't, they are flagged with this code.

In [33]:
LDStatusdf= VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'].isin(LDStatus)]
LDSDateNull = LDStatusdf[LDStatusdf['Date Since'].isna()]

This next section is built off of LDStatusdf, which was built off of LDStatus. I didn't put them together because this one is checking for something different. 

We are checking for the records in the column 'Current Discharge Status' that are equal to 'Vendor not seeking discharge'. Once we have those, we are checking to see if the 'More than 6 months since status date?' is greater than or equal to 365. 

For all of the records that are properly selected (LDStatusdf), if the vendor is not seeking discharge, we'd like to know how long its been since their status date. If it is longer than a year, we'd like to know why. 

In [34]:
LDStemp = LDStatusdf[LDStatusdf['Current Discharge Status'] == 'Vendor not seeking discharge']
LDS_NoDIS_1YR = LDStemp[LDStemp['More than 6 months since status date?'] >= 365]

This following cell checks for cases that have a Yes in the Petition Filed column, yet do not have a Petition Date (checked with the .isna() function). 

NOTICE there is a couple of spaces after Filed in the column name. 'Petition Filed  ' is not the same as 'Petition Filed'. That is an error with the excel file. It could be fixed with a function, such as str.strip(), but I figured it might be better to leave it like this for educational purposes. 

In [35]:
YPetition = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Petition Filed  '] == 'Yes']
YPetitionNA = YPetition[YPetition['Petition Date'].isna()]

YPetitionUrgent located all of the rows in the columns Hearing Date to Comments for Current Discharge Status that are empty. If a client has filed a petition, but they do not have discharge information, we'd like to know what is going on with that discharge. 

In [36]:
YPetitionUrgent = YPetition[YPetition.loc[: , 'Hearing Date' : 'Comments for Current Discharge Status'].isna().all(axis = 1)]

Similar to YPetition, this shows all cases that have NOT filed a Petition and have a value in the Petition Date (checked using .notna())
Again, note that Petition Filed has two spaces after its name. 

In [37]:
NPetition = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Petition Filed  '] == 'No']
NPetitionVAL = NPetition[NPetition['Petition Date'].notna()]

The next cell focuses on clients who have Clt discharged as a Current Discharge Status, yet have an empty value for the Discharge order date. 

In [38]:
CD = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Current Discharge Status'] == 'Clt discharged']
CD_Null_Date = CD[CD['D/C order Date'].isna()]

Cases where they have selected Out of Jurisdiction as their Living/Domicile Status, but have an empty current discharge status. 
The question we are focusing on is: How are they out of NYC, yet are not discharged?

In [39]:
Jurisdictiondf = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'Out of Jurisdiction']
JCDS_Null = Jurisdictiondf[Jurisdictiondf['Current Discharge Status'].isna()]

Checking for the condition 'Nursing Home-Temp' in the Living/Domicile Status column. 
Followed by a search for all cases that have an empty record for Days since HRA's Approval. 
Finally, checking if the client has more than 6 months, yet less than 1 year for 'More than 6 months since status date?'

If the client is temporarily in the nursing home, and HRA has not approved their discharge, this code highlights cases where it has been more than 6 months and less than 1 year since they were placed. That is an abnormally large amount of time, which merits a red flag.

In [40]:
NHT = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'Nursing Home-Temp']
NHT_HRA_NULL = NHT[NHT["Days since HRA's Approval"].isna()]
NHT_DATE_180 = NHT_HRA_NULL[(NHT_HRA_NULL['More than 6 months since status date?'] >= 180) & (NHT_HRA_NULL['More than 6 months since status date?'] < 365)]

This code is based off of the NHT_HRA_NULL section above, and is checking for the 'More than 6 months since status date?' is greater or equal to 1 year (unlike the code above that checks for 180 to 365 days). 

Once we have collected these extreme records, using .loc and slicing, we are checking for empty discharge information, because then we know that there is no work being done on these clients, and we definitely need to investigate. 

In [41]:
NHT_DATE_365 = NHT_HRA_NULL[NHT_HRA_NULL['More than 6 months since status date?'] >= 365]
NHT_365_NO_PROGRESS = NHT_DATE_365[NHT_DATE_365.loc[:, 'Date Discharge Request Submitted to HRA':'Comments for Current Discharge Status'].isna().all(axis = 1)]

This code highlights the instances where the Judge denied Discharge is the value in the Current Discharge Status column. Once we have those records, we are checking for empty values in the Comments for Current Discharge Status. 

If the Judge denied discharge, the vendor is supposed to explain why it was denied in the comments section. If it is empty, there is no explanation, so we need to ask them what is going on.

In [42]:
DENIED = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Current Discharge Status'] == 'Judge denied D/C']
DENIED_NA = DENIED[DENIED['Comments for Current Discharge Status'].isna()]

We begin the next cell by checking if 'HRA Approved (Y/N)' is equal to Yes for the entries in the excel sheet. 
Once we have this fundamental DataFrame, we use .loc and slicing to check for empty entries for the columns Petition Filed to D/C order Date. 
After that, we begin checking for the Day's since HRA's Approval using different conditions. 
HRA120 checks between 60 and 120 days.
HRA180 checks for 120-180 days.
HRA365 checks for 180-365 days. 
HRA_SUBMIT_HRA checks for cases that have a Submit D/C Request to HRA entry and follows the criteria indiacted above before checking for the different Days since HRA's Approval conditions. 

In [43]:
HRA_YES = VENDOR_LIST_FULL[VENDOR_LIST_FULL["HRA Approved (Y/N)"] == 'Yes']
HRA = HRA_YES[HRA_YES.loc[:, 'Petition Filed  ':'D/C order Date'].isna().all(axis = 1)]
HRA120 = HRA[(HRA["Days since HRA's Approval"] >= 60) & (HRA["Days since HRA's Approval"]<= 120)]
#YES HRA Approval and days since 60 - 120 and nothing columns Z - AC
HRA180 = HRA[(HRA["Days since HRA's Approval"] >= 120) & (HRA["Days since HRA's Approval"]<= 180)]
#YES HRA Approval and days since 120 - 180 and nothing columns Z - AC
HRA365 = HRA[(HRA["Days since HRA's Approval"] >= 180) & (HRA["Days since HRA's Approval"]<= 365)]
#YES HRA Approval and days since 180 - 365 and nothing columns Z - AC
HRA_SUBMIT_HRA = HRA[HRA['Current Discharge Status'] == 'Submit D/C Request to HRA']
#YRS HRA Approval AND CDS = Submit DC Request to HRA

This chunk of code brute forces selection criteria and assigns it to different variables. 
These are the collection of different Current Discharge Status, and will be used by the office as a filter. 

In [44]:
CLT_COMMUNITY = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Current Discharge Status'] == 'Clt back in the community']
CLT_COMMUNITY_6MNTHS = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Current Discharge Status'] == 'Clt scheduled to return to the community within 6 months of admission']
DC_REQUEST = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Current Discharge Status'] == 'Submit D/C Request to HRA']
JUDGE_DENIED_DC = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Current Discharge Status'] == 'Judge denied D/C']
NO_SGUARDIAN = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Current Discharge Status'] == 'Successor guardian not yet commissioned']
VENDOR_NO_DISCHARGE = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Current Discharge Status'] == 'Vendor not seeking discharge']
CLT_RETURN_NYC = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Current Discharge Status'] == 'Clt returned back to NYC']
SGUARDIAN_FA = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Current Discharge Status'] == 'Success guardian appointed FA to be submitted']
CLT_DISCHARGED = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Current Discharge Status'] == 'Clt discharged']
PENDING_COURT_APP = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Current Discharge Status'] == 'Pending Court Approval']

Similar to above, there are various variables that each represent the different types of Living/Domicile Status.

In [45]:
DECEASED = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'Deceased']
ADULTHOME = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'Adult Home']
ASSISTED_LIVING = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'Assisted Living']
SUPPORTIVE_HOUSING = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'Supporting Housing']
HOSPITAL = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'Hospital']
SHELTER = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'Shelter']
NURSING_HOME_PERM = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'Nursing Home-Perm']
NURSING_HOME_TEMP = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'Nursing Home-Temp']
STREET_HOMELESS = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'Street Homeless']
INCARCERATED = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'Incarcerated']
RESIDES_COMMUNITY = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'Resides in Community']
WHEREABOUTS_UNK = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'Whereabouts unknown']
OUT_JURIS = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'Out Of Jurisdiction']
SUBSTANCE_ABUSE = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'Substance abuse facility']
OPWDD_FACILITY = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'OPWDD facility']
CONGREGATE_CARE = VENDOR_LIST_FULL[VENDOR_LIST_FULL['Living/Domicile Status'] == 'Congregate Care Facility']  

This final chunk of code uses the function that we made in the beginning to export all of the variables that we made into an excel sheet, and have every entry in a different sheet. This follows the exact format that was mentioned in the beginning. There is a brief comment below every entry that explains what is happening and why it is used. 

The sheets are in the order that was requested by Gili's department, and covers all of the information that they wanted. Until now, they've been satisfied, and haven't requested more information.

Future work will include checking for duplicate values between the record made by this project and records from future months. If there are duplicate values, that means the vendor is not updating their clients as they should.

In [ ]:
    #To be updated by Vendor
    
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', Missing_Demo, sheet_name = 'Missing_Demo')
    #All cases with missing blue category

    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', INVALID_SOCIAL, sheet_name = 'Invalid_Social')
    #Social less than 9 digits

    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', NO_SOCIAL, sheet_name = 'No_Social')
    #Missing Social

    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', LDS_DECEASED_SUB, sheet_name = 'Decd DC to HRA')
    #Cases Living / Domicile = Deceased AND Current Discharge Status. = Submit DC request to HRA

    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', LDS_ERROR, sheet_name = 'LDS_ERROR')
    #Cases with flawed Living/Domicile Status (NaN, typos, not in proper list)

    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', CDS_ERROR, sheet_name = 'CDS_ERROR')
    #Cases with flawed Current Discharge Status (I think they're all NaN. dropna() = 0 rows)


    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', LDSDateNull, sheet_name = 'Date Since Required')
    #Cases where LDS != Residing in Community and Date Since is null
    
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', YPetitionNA, sheet_name = 'Y Petition N Date')
    #Yes to petition filed, yet has no petition date

    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', NPetitionVAL, sheet_name = 'N Petition Y Date')
    #No to petition filed, yet has a petition date

        
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', YPetitionUrgent, sheet_name = 'Y Petition No DC Info')
    #Yes Petition with nothing in columns AB-AE (Hearing Date to Comments for CDS)
        
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', CD_Null_Date, sheet_name = 'CDS DC No DC Date')
    #Clients discharged without discharge date

    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', JCDS_Null, sheet_name = 'Out of Juris No CDS')
    #All Cases out of Jurisdiction that don't have a discharge status
    
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', LDS180_FOCUS, sheet_name = 'INS 180 to 365 No DC Info')
    #cases filtered that have 180-365 status date and have nothing in Z-AE

    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', LDS365_FOCUS, sheet_name = 'Ind 365 No DC Info')
    #cases filtered that have >=365 status date and have nothing in Z-AE

    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', HRA_SUBMIT_HRA, sheet_name = 'HRA App Y CDC Submit HRA')
    #YRS HRA Approval AND CDS = Submit DC Request to HRA

    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', DENIED_NA, sheet_name = 'CDS JDG and No Comment')
    #Judge Denied CDS and Comments are empty
    


#INTERNAL REVIEW NEEDED
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', VALUE_COUNT, sheet_name = 'Value_Count')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', LIST_TYPE, sheet_name = 'List Type Count')
    
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', MONTH_ERROR, sheet_name = 'Month_Error')
    #Cases that have #VALUE! in Month column
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', LDS_DECEASED, sheet_name = 'LDS_DECEASED')
    #Cases where LDS == Deceased

    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', LDS_removed, sheet_name = 'Inst Clts')
    #Living Domicile Status Without Deceased / Homeless / whereabouts / jurisdiction / resides in community

    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', LDS180, sheet_name = 'Inst more than 180 days')
    #cases filtered that have >180 status date

    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', LDS365, sheet_name = 'Inst more than 365 days')
    #cases filtered that have >365 status date
    
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', NHT_DATE_180, sheet_name = 'NHT more 180 No HRA Appr')
    #Nursing Home Temp Date Since is between 180 and 365 and days since HRA approval = NULL
    
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', NHT_365_NO_PROGRESS, sheet_name = 'NHT more 365 No HRA DC Info')
    #Nursing Home Temp Date Since is greater than 365 and days since HRA approval = NULL AND nothing in columns V-AE
    
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', LDS_NO_DISCHARGE, sheet_name = 'LDS Deceased No DC Info')
    #Client is deceased and nothing in columns Z-AE
    
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', LDS_NoDIS_1YR, sheet_name = 'Date since more 1yr CDC VNSD')
    #Anything other than resides in the community and date since is greater than 1 year and CDS = vendor not seeking discharge

    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', HRA120, sheet_name = 'HRA Y more 60 No DC Info')
    #YES HRA Approval and days since 60 - 120 and nothing columns Z - AC
    
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', HRA180, sheet_name = 'HRA Y more 180 No DC Info')
    #YES HRA Approval and days since 120 - 180 and nothing columns Z - AC
    
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', HRA365, sheet_name = 'HRA Y more 365 No DC Info')
    #YES HRA Approval and days since 180 - 365 and nothing columns Z - AC
    
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', HRA_DECEASED_NULL, sheet_name = 'HRA_DECEASED_BLANK')
    #Cases where HRA Approval is NULL and the client is not deceased.
    

    #LDS
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', DECEASED, sheet_name = 'DECEASED')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', ADULTHOME, sheet_name = 'ADULTHOME')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', ASSISTED_LIVING, sheet_name = 'ASSISTED_LIVING')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', SUPPORTIVE_HOUSING, sheet_name = 'SUPPORTIVE_HOUSING')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', HOSPITAL, sheet_name = 'HOSPITAL')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', SHELTER, sheet_name = 'SHELTER')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', NURSING_HOME_PERM, sheet_name = 'NURSING_HOME_PERM')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', NURSING_HOME_TEMP, sheet_name = 'NURSING_HOME_TEMP')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', STREET_HOMELESS, sheet_name = 'STREET_HOMELESS')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', INCARCERATED, sheet_name = 'INCARCERATED')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', RESIDES_COMMUNITY, sheet_name = 'RESIDES_COMMUNITY')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', WHEREABOUTS_UNK, sheet_name = 'WHEREABOUTS_UNK')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', OUT_JURIS, sheet_name = 'OUT_JURIS')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', SUBSTANCE_ABUSE, sheet_name = 'SUBSTANCE_ABUSE')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', OPWDD_FACILITY, sheet_name = 'OPWDD_FACILITY')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', CONGREGATE_CARE, sheet_name = 'CONGREGATE_CARE')

    #CDS
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', CLT_COMMUNITY, sheet_name = 'CLT_COMMUNITY')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', CLT_COMMUNITY_6MNTHS, sheet_name = 'CLT_COMMUNITY_6MNTHS')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', DC_REQUEST, sheet_name = 'DC_REQUEST')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', JUDGE_DENIED_DC, sheet_name = 'JUDGE_DENIED_DC')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', NO_SGUARDIAN, sheet_name = 'NO_SGUARDIAN')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', VENDOR_NO_DISCHARGE, sheet_name = 'VENDOR_NO_DISCHARGE')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', CLT_RETURN_NYC, sheet_name = 'CLT_RETURN_NYC')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', SGUARDIAN_FA, sheet_name = 'SGUARDIAN_FA')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', CLT_DISCHARGED, sheet_name = 'CLT_DISCHARGED')
    append_df_to_excel('/Users/Carlos/Desktop/Vendor_List_Exceptions.xlsx', PENDING_COURT_APP, sheet_name = 'PENDING_COURT_APP')